In [1]:
#importing all required modules
import findspark
findspark.init()
findspark.find()
import pyspark
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
conf = pyspark.SparkConf().setAppName('appName').setMaster('local')
sc = pyspark.SparkContext.getOrCreate(conf=conf)
spark = SparkSession(sc)
from pyspark.sql import functions as F
from pyspark.ml.feature import Tokenizer, RegexTokenizer
from pyspark.sql.functions import col, udf
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StopWordsRemover

In [2]:
#Reading business data from json file
businessdata = spark.read.json("yelp_academic_dataset_business.json")
businessdata.show()

+--------------------+--------------------+--------------------+--------------------+------------------+--------------------+-------+-------------+---------------+--------------------+-----------+------------+-----+-----+
|             address|          attributes|         business_id|          categories|              city|               hours|is_open|     latitude|      longitude|                name|postal_code|review_count|stars|state|
+--------------------+--------------------+--------------------+--------------------+------------------+--------------------+-------+-------------+---------------+--------------------+-----------+------------+-----+-----+
|2818 E Camino Ace...|[,,,,,,,,,,,,,,,,...|1SWheh84yJXfytovI...|   Golf, Active Life|           Phoenix|                null|      0|   33.5221425|   -112.0184807|Arizona Biltmore ...|      85016|           5|  3.0|   AZ|
|30 Eglinton Avenue W|[,, u'full_bar', ...|QXAEGFB4oINsVuTFx...|Specialty Food, R...|       Mississauga|[9:0-1:0

In [4]:
#Reading review data from json file
reviewdata = spark.read.json("yelp_academic_dataset_review.json")
reviewdata.show()

+--------------------+----+-------------------+-----+--------------------+-----+--------------------+------+--------------------+
|         business_id|cool|               date|funny|           review_id|stars|                text|useful|             user_id|
+--------------------+----+-------------------+-----+--------------------+-----+--------------------+------+--------------------+
|ujmEBvifdJM6h6RLv...|   0|2013-05-07 04:34:36|    1|Q1sbwvVQXV2734tPg...|  1.0|Total bill for th...|     6|hG7b0MtEbXx5QzbzE...|
|NZnhc2sEQy3RmzKTZ...|   0|2017-01-14 21:30:33|    0|GJXCdrto3ASJOqKeV...|  5.0|I *adore* Travis ...|     0|yXQM5uF2jS6es16SJ...|
|WTqjgwHlXbSFevF32...|   0|2016-11-09 20:09:03|    0|2TzJjDVDEuAW6MR5V...|  5.0|I have to say tha...|     3|n6-Gk65cPZL6Uz8qR...|
|ikCg8xy5JIg_NGPx-...|   0|2018-01-09 20:56:38|    0|yi0R0Ugj_xUx_Nek0...|  5.0|Went in for a lun...|     0|dacAIZ6fTM6mqwW5u...|
|b1b1eb3uo-w561D0Z...|   0|2018-01-30 23:07:38|    0|11a8sVPMUFtaC7_AB...|  1.0|Today was 

In [6]:
#selecting only related businesses(Active life - gym,yoga, martial arts, dance studios) from business dataset 
businessdata.createOrReplaceTempView("Business")
fitnessbusinessTable = spark.sql("select business_id, name, address, city, state, postal_code, latitude, longitude, stars as overstars, review_count, categories from Business where categories like '%Active Life%'")

reviewdata.createOrReplaceTempView("Reviews")
reviewTable = spark.sql("select * from Reviews")

#Joining business wtih reviews dataset
fitnessbusinessreviewdata = fitnessbusinessTable.join(reviewTable, 'business_id')
fitnessbusinessreviewdata.createOrReplaceTempView("FBusinessReviewData")

# fitnessbusinessreviewdata.printSchema()

# printing only required elements
fitnessbusinessreviewdata.select("business_id", "name", "city", "state", "categories", "stars", "text").show()

+--------------------+--------------------+---------------+-----+--------------------+-----+--------------------+
|         business_id|                name|           city|state|          categories|stars|                text|
+--------------------+--------------------+---------------+-----+--------------------+-----+--------------------+
|2sTYUvBkn9qvQAH3A...|North Olmsted Gol...|       Westlake|   OH|Golf Lessons, Act...|  1.0|Having lived on t...|
|2sTYUvBkn9qvQAH3A...|North Olmsted Gol...|       Westlake|   OH|Golf Lessons, Act...|  1.0|We did not really...|
|2sTYUvBkn9qvQAH3A...|North Olmsted Gol...|       Westlake|   OH|Golf Lessons, Act...|  3.0|BLUF: A well main...|
|4Tk1cIKGE19FTHjlP...|    Top Line Defense|      Las Vegas|   NV|Gun/Rifle Ranges,...|  5.0|Awesome!!!! Herb ...|
|4Tk1cIKGE19FTHjlP...|    Top Line Defense|      Las Vegas|   NV|Gun/Rifle Ranges,...|  5.0|I decided to get ...|
|4Tk1cIKGE19FTHjlP...|    Top Line Defense|      Las Vegas|   NV|Gun/Rifle Ranges,...|  

In [7]:
#remove null data or any abnormal data
fbusinessreviewdatafiltered = spark.sql("select * from FBusinessReviewData where business_id IS NOT NULL And review_id IS NOT NULL And city IS NOT NULL And categories IS NOT NULL And text IS NOT NULL")
fbusinessreviewdatafiltered.createOrReplaceTempView("FBusinessReviewDataF")

# more filtering depending on studio types. Since we have considered only certain studio types this filter is required else no need for this filter. Playground, soccer and basketall to be consider as one(Outdoor) while the others to be considered themselves.    
fbusinessreviewdatafr = spark.sql("select * from FBusinessReviewDataF where categories Like '%Gyms%' OR categories Like '%Martial Arts%' OR categories Like '%Dance Studios%' OR categories Like '%Yoga%' OR categories Like '%Pilates%' OR categories Like '%Swimming%' OR categories Like '%Massage%' OR categories Like '%Playground%' OR categories Like '%Soccer%' OR categories Like '%Basketball%' ")

#Segregating data in two types positive and negative datasets depending on comment rating given by the user. We have onsidered 3 stars to be part of negative dataset since our datset was inclined to the positive side(more positive comments than negative comments), thus to balance it out. However we can considered 3 stars rating data to be neutral and can have one more datatye for processing more accurately.
fbusinessreviewdatafrP = fbusinessreviewdatafr.select(fbusinessreviewdatafr.columns).where(F.col('stars') > 3 )
fbusinessreviewdatafrN = fbusinessreviewdatafr.select(fbusinessreviewdatafr.columns).where(F.col('stars') <= 3 )

fbusinessreviewdatafrP.createOrReplaceTempView("FBusinessReviewDataPos")
fbusinessreviewdatafrN.createOrReplaceTempView("FBusinessReviewDataNeg")


In [8]:
# all postive reviews dataset (review with 4 and 5 stars)
allPreviews = fbusinessreviewdatafr.select("text").where(F.col('stars') > 3 )
allPreviews.show()

+--------------------+
|                text|
+--------------------+
|Awesome!!!! Herb ...|
|I decided to get ...|
|My husband and I ...|
|Recently did my N...|
|Just passed my CC...|
|AMAZING! i was a ...|
|10 Mar 2017 Updat...|
|What an incredibl...|
|Before arriving f...|
|My husband and I ...|
|Je n'y suis allée...|
|Ce n'est pas le p...|
|The most lovely p...|
|En ces temps de c...|
|Mon parc et endro...|
|Excellent parc po...|
|Though not grand ...|
|Un de mes parcs p...|
|I love Jarry Park...|
|I love this park ...|
+--------------------+
only showing top 20 rows



In [9]:
# all negative reviews dataset (review with 3, 2 and 1 stars)
allNreviews = fbusinessreviewdatafr.select("text").where(F.col('stars') <= 3 )
allNreviews.show()

+--------------------+
|                text|
+--------------------+
|nice enough park ...|
|I was so excited ...|
|This place is a g...|
|My friend had tol...|
|We still really l...|
|I really like the...|
|At first, this fa...|
|Been in AZ for ab...|
|It's not the clas...|
|I really like the...|
|Took level 1 pole...|
|I took a class at...|
|I took several cl...|
|This place is pro...|
|I went this past ...|
|Was great until t...|
|Our instructor Br...|
|This place sucks!...|
|I took a split fo...|
|I really like thi...|
+--------------------+
only showing top 20 rows



In [10]:
# Using regextokenizer to create tokens of each word. We are using unigram model, hence one word is one token, We can further improve the analysis by making use of bigrams and trigrams 
regexTokenizer = RegexTokenizer(inputCol="text", outputCol="words", pattern="\\W")

regexTokenizedP = regexTokenizer.transform(allPreviews)
regexTokenizedP.select("text", "words").show(truncate=False)

regexTokenizedN = regexTokenizer.transform(allNreviews)
# regexTokenizedN.select("text", "words").show(truncate=False)


+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [11]:
# Using stopwords remover to remove common english words like a, an the, I etc
remover = StopWordsRemover(inputCol="words", outputCol="filtered")

basicwordsfilteredP = remover.transform(regexTokenizedP)
basicwordsfilteredP.show(truncate=False)

basicwordsfilteredN = remover.transform(regexTokenizedN)
# basicwordsfilteredN.show(truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [12]:
# reqwords are concatenated as a string for further processing
reqwordsstringP = basicwordsfilteredP.withColumn("strform", F.concat_ws(" ", "filtered"))
reqwordsstringP.show(truncate=False)

reqwordsstringN = basicwordsfilteredN.withColumn("strform", F.concat_ws(" ", "filtered"))
# reqwordsstringN.show(truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [13]:
# aggregating each tokens(word) to get their total count. So we get a count of a word in postive context (ie how many time the word is used in positive context). Similarly we get a negative context words and their count. 
alldatawordscountP = reqwordsstringP.withColumn('word', F.explode(F.split(F.col('strform'), ' '))).groupBy('word').count().sort('count', ascending=False)
alldatawordscountP.show(100)
# alldatawordscountP.count()

+-----------+-----+
|       word|count|
+-----------+-----+
|      great|37198|
|        gym|27895|
|      place|26598|
|      class|26294|
|        get|24436|
|    classes|24177|
|       like|23410|
|       time|22201|
|        one|21845|
|       love|19791|
|     really|19040|
|         ve|17907|
|       yoga|17340|
|       also|17169|
|       good|16351|
|         go|16057|
|      staff|15885|
|       pool|15759|
|     always|14476|
|     people|14386|
|      clean|13801|
|       nice|13658|
|     studio|13457|
|          m|13370|
|       best|13321|
|    workout|13319|
|       back|13132|
|   friendly|13059|
|       well|13032|
|       work|13003|
|        day|12587|
|       room|12215|
|      first|12024|
|       kids|11826|
|       feel|11225|
|        fun|11197|
|    fitness|11168|
|    amazing|11107|
|       even|10867|
|       area|10550|
|  recommend|10384|
|      going|10375|
|       much|10327|
|        new|10160|
|      never| 9710|
|      every| 9402|
|         re| 9081|


In [14]:
alldatawordscountN = reqwordsstringN.withColumn('word', F.explode(F.split(F.col('strform'), ' '))).groupBy('word').count().sort('count', ascending=False)
alldatawordscountN.show(100)
# alldatawordscountN.count()

+----------+-----+
|      word|count|
+----------+-----+
|       gym|18905|
|       get|15289|
|       one|14854|
|      like|13733|
|      time|12587|
|     place|12322|
|        go|10824|
|    people|10094|
|      back| 9276|
|     class| 8901|
|      even| 8878|
|      pool| 8692|
|      room| 8442|
|      told| 8355|
|     staff| 7895|
|      good| 7867|
|    really| 7812|
|membership| 7739|
|     never| 7707|
|         m| 7566|
|       day| 7115|
|      said| 7013|
|      also| 6810|
|   service| 6538|
|        ve| 6537|
|      didn| 6510|
|   classes| 6479|
|     going| 6385|
|         2| 6027|
|      work| 5946|
|        us| 5908|
|   fitness| 5860|
|     great| 5823|
|      nice| 5769|
|      went| 5730|
|     month| 5727|
|     first| 5711|
|       got| 5613|
|      want| 5511|
| equipment| 5320|
|      know| 5303|
|      area| 5271|
|  location| 5269|
|      kids| 5167|
|     money| 5112|
|       way| 5007|
|         3| 5006|
|      much| 4899|
|    around| 4872|
|     front|

In [15]:
# further processing of top 1000 common positive and negative words to generate a score for each word. A word can occur in both context and will have a different positive and negative score.
# Select top 1000 positive words and negative words.Also filtering words whose length is less than 3 since they will be common english words.
alldatawordscountP.createOrReplaceTempView("AllDataWordsCountP")
alldatawordscountPtop = spark.sql("select word, count as totalocc from AllDataWordsCountP where char_length(word) > 2 limit 1000")
alldatawordscountPtop.createOrReplaceTempView("AllDataWordsCountPTop")
# alldatawordscountPtop.count()

alldatawordscountN.createOrReplaceTempView("AllDataWordsCountN")
alldatawordscountNtop = spark.sql("select word, count as totalocc from AllDataWordsCountN where char_length(word) > 2 limit 1000")
alldatawordscountNtop.createOrReplaceTempView("AllDataWordsCountNTop")
# alldatawordscountNtop.count()

In [16]:
# Calculate the sum of the occurences of this top 1000 words and get a totalpsum and totalnsum
allPdatatotaloccsum = spark.sql("select sum(totalocc) as totaloccsum from AllDataWordsCountPTop")
allPdatatotaloccsum = allPdatatotaloccsum.take(1)
print("positive word sum = "+ str(allPdatatotaloccsum[0][0]))

allNdatatotaloccsum = spark.sql("select sum(totalocc) as totaloccsum from AllDataWordsCountNTop")
allNdatatotaloccsum = allNdatatotaloccsum.take(1)
print("negative word sum = "+ str(allNdatatotaloccsum[0][0]))


positive word sum = 2503460
negative word sum = 1409549


In [17]:
#divide each wordcount with totalpsum(positive word sum) for positive context and with totalnsum(negative word sum) for negative context to get a relative score for each word in positive and negative context.
allPdatawordpoints = spark.sql("select word, totalocc, totalocc/"+str(allPdatatotaloccsum[0][0])+ " as wordpoints from AllDataWordsCountPTop")
allPdatawordpoints.show(100)

+-----------+--------+--------------------+
|       word|totalocc|          wordpoints|
+-----------+--------+--------------------+
|      great|   37198|0.014858635648262805|
|        gym|   27895| 0.01114257867111917|
|      place|   26598| 0.01062449569795403|
|      class|   26294|0.010503063759756497|
|        get|   24436|0.009760890926957092|
|    classes|   24177|0.009657434111190112|
|       like|   23410|0.009351058135540413|
|       time|   22201|0.008868126512906138|
|        one|   21845|0.008725923322122183|
|       love|   19791|0.007905458844958578|
|     really|   19040|0.007605474023950852|
|       yoga|   17340|0.006926413843240955|
|       also|   17169|0.006858108378004841|
|       good|   16351|0.006531360596933...|
|      staff|   15885|0.006345218217986307|
|       pool|   15759| 0.00629488787518075|
|     always|   14476|0.005782397162327339|
|     people|   14386|0.005746446917466227|
|      clean|   13801|0.005512770325868997|
|       nice|   13658|0.00545564

In [18]:
#same as we did for positive context words.
allNdatawordpoints = spark.sql("select word, totalocc, totalocc/"+str(allNdatatotaloccsum[0][0])+ " as wordpoints from AllDataWordsCountNTop")
allNdatawordpoints.show(100)

+----------+--------+--------------------+
|      word|totalocc|          wordpoints|
+----------+--------+--------------------+
|       gym|   18905|0.013412091385258689|
|       get|   15289|0.010846731826988632|
|       one|   14854| 0.01053812247747329|
|      like|   13733|0.009742832636538353|
|      time|   12587| 0.00892980662609104|
|     place|   12322|0.008741803229259855|
|    people|   10094|0.007161155802316...|
|      back|    9276| 0.00658082833587197|
|     class|    8901|0.006314785793186...|
|      even|    8878|0.006298468517234946|
|      pool|    8692|0.006166511416062869|
|      room|    8442|0.005989149720939109|
|      told|    8355|0.005927427851036041|
|     staff|    7895|0.005601082332008323|
|      good|    7867|0.005581217822154462|
|    really|    7812|0.005542198249227235|
|membership|    7739|0.005490408634251097|
|     never|    7707|0.005467706337275...|
|       day|    7115|0.005047713843222194|
|      said|    7013|  0.0049753502716117|
|      also

In [19]:
finalscores = []
avgscores = []

In [173]:
###############################
# This could be in for loop. Here we can replace cityName with the required city and fitnessStudioType with the required fitnessStudioType to find a score for that cityStudio.
# Then we can compare the score with other citystudio types and can come to a conclusion which studio is preffered in which city and all.  
# For calculating score the same procedure is done as we have till now and have taken top 500 positive and top 500 negative words for a particular CityStudio Type.
# Then we compared it with original score of the word and multiply them to get a final score of the word. Then do summation of such 100 words for both positive and negative context and genrate a final score for thatCityStudio.
# Then we do average of the both and thus we can determine which studio is preffered more in which city. 
# Also top 100 positive and top 100 negative words can be used to improve the studio in that city type. 

cityName = 'Las Vegas' #Las Vegas, Tempe, Phoenix, Cleveland, Scottsdale, Pittsburgh
fitnessStudioType = 'Gym' #Gym, Martial Arts, Yoga, Dance Studios

citystudioBRposData = spark.sql("select text from FBusinessReviewDataPos where categories Like '%"+fitnessStudioType+"%' AND city = '"+cityName+"'")
citystudioBRnegData = spark.sql("select text from FBusinessReviewDataNeg where categories Like '%"+fitnessStudioType+"%' AND city = '"+cityName+"'")

In [174]:
regexTokenizer = RegexTokenizer(inputCol="text", outputCol="words", pattern="\\W")
regexTokenizedCSP = regexTokenizer.transform(citystudioBRposData)
regexTokenizedCSN = regexTokenizer.transform(citystudioBRnegData)

In [175]:
remover = StopWordsRemover(inputCol="words", outputCol="filtered")
basicwordsfilteredCSP = remover.transform(regexTokenizedCSP)
basicwordsfilteredCSN = remover.transform(regexTokenizedCSN)

In [176]:
reqwordsstringCSP = basicwordsfilteredCSP.withColumn("strform", F.concat_ws(" ", "filtered"))
reqwordsstringCSN = basicwordsfilteredCSN.withColumn("strform", F.concat_ws(" ", "filtered"))

In [177]:
alldatawordscountCSP = reqwordsstringCSP.withColumn('word', F.explode(F.split(F.col('strform'), ' '))).groupBy('word').count().sort('count', ascending=False)
alldatawordscountCSN = reqwordsstringCSN.withColumn('word', F.explode(F.split(F.col('strform'), ' '))).groupBy('word').count().sort('count', ascending=False)

In [178]:
alldatawordscountCSP.createOrReplaceTempView("AllDataWordsCountCSP")
alldatawordscountCSPtop = spark.sql("select word, count as totalocc from AllDataWordsCountCSP where char_length(word) > 2 limit 500")
alldatawordscountCSPtop.createOrReplaceTempView("AllDataWordsCountCSPTop")

alldatawordscountCSN.createOrReplaceTempView("AllDataWordsCountCSN")
alldatawordscountCSNtop = spark.sql("select word, count as totalocc from AllDataWordsCountCSN where char_length(word) > 2 limit 500")
alldatawordscountCSNtop.createOrReplaceTempView("AllDataWordsCountCSNTop")

In [179]:
allCSPdatatotaloccsum = spark.sql("select sum(totalocc) as totaloccsum from AllDataWordsCountCSPTop")
allCSPdatatotaloccsum = allCSPdatatotaloccsum.take(1)

allCSNdatatotaloccsum = spark.sql("select sum(totalocc) as totaloccsum from AllDataWordsCountCSNTop")
allCSNdatatotaloccsum = allCSNdatatotaloccsum.take(1)

In [180]:
allCSPdatawordpoints = spark.sql("select word as word, totalocc as csptotalocc, totalocc/"+str(allCSPdatatotaloccsum[0][0])+ " as cspwordpoints from AllDataWordsCountCSPTop")

In [181]:
allCSNdatawordpoints = spark.sql("select word as word, totalocc as csntotalocc, totalocc/"+str(allCSNdatatotaloccsum[0][0])+ " as csnwordpoints from AllDataWordsCountCSNTop")

In [182]:
allCSPDWPO = allCSPdatawordpoints.join(allPdatawordpoints, on=['word'], how='inner')
# allCSPDWPO.show(100)

In [183]:
allCSPDWPOL = allCSPDWPO.select(allCSPDWPO.columns).limit(100)
allCSPDWPOL.createOrReplaceTempView("AllCSPosDataWPOL")
allCSPDWPOLM = spark.sql("select word, cspwordpoints, wordpoints,  Round((cspwordpoints * wordpoints * 10000), 5) as totalpoints from AllCSPosDataWPOL")
allCSPDWPOLM.show(100)

+-----------+--------------------+--------------------+-----------+
|       word|       cspwordpoints|          wordpoints|totalpoints|
+-----------+--------------------+--------------------+-----------+
|      great| 0.01612340336427724|0.014858635648262805|    2.39572|
|        gym| 0.02527395826062679| 0.01114257867111917|    2.81617|
|      place|0.011394569693585538| 0.01062449569795403|    1.21062|
|      class| 0.01000558386263698|0.010503063759756497|    1.05089|
|        get|0.012954561317791583|0.009760890926957092|    1.26448|
|    classes|  0.0089900188455364|0.009657434111190112|    0.86821|
|       like|0.012322886856983318|0.009351058135540413|    1.15232|
|       time| 0.01000558386263698|0.008868126512906138|    0.88731|
|        one|0.010612828924408459|0.008725923322122183|    0.92607|
|       love|0.009373909401828716|0.007905458844958578|    0.74105|
|     really|0.008340894813987577|0.007605474023950852|    0.63436|
|       yoga|0.002100928317163...|0.006926413843

In [184]:
allCSPDWPOLM.createOrReplaceTempView("AllCSPosDataWPOLM")
CSPosDataFinalPoint = spark.sql("select sum(totalpoints) as finalpoints from AllCSPosDataWPOLM")
CSPosDataFinalPoint = CSPosDataFinalPoint.take(1)
CSPosScore = round(CSPosDataFinalPoint[0][0], 3)
print(CSPosScore)


28.627


In [185]:
allCSNDWPO = allCSNdatawordpoints.join(allNdatawordpoints, on=['word'], how='inner')
# allCSPDWPO.show(100)
allCSNDWPOL = allCSNDWPO.select(allCSNDWPO.columns).limit(100)
allCSNDWPOL.createOrReplaceTempView("AllCSNegDataWPOL")
allCSNDWPOLM = spark.sql("select word, csnwordpoints, wordpoints,  Round((csnwordpoints * wordpoints * 10000), 5) as totalpoints from AllCSNegDataWPOL")
allCSNDWPOLM.show(100)

+----------+--------------------+--------------------+-----------+
|      word|       csnwordpoints|          wordpoints|totalpoints|
+----------+--------------------+--------------------+-----------+
|       gym|0.020680436011645296|0.013412091385258689|    2.77368|
|       get|0.013531578085289627|0.010846731826988632|    1.46773|
|       one|0.012892249327648063| 0.01053812247747329|     1.3586|
|      like|0.012279339113710696|0.009742832636538353|    1.19636|
|      time| 0.01116447656939359| 0.00892980662609104|    0.99697|
|     place|0.008765672801052515|0.008741803229259855|    0.76628|
|    people|0.008258436761931935|0.007161155802316...|     0.5914|
|      back|0.007862158606368982| 0.00658082833587197|     0.5174|
|     class|0.004031469769260439|0.006314785793186...|    0.25458|
|      even|0.007782902975256392|0.006298468517234946|     0.4902|
|      pool|0.002340682972191841|0.006166511416062869|    0.14434|
|      room|0.008929467772018536|0.005989149720939109|     0.5

In [186]:
allCSNDWPOLM.createOrReplaceTempView("AllCSNegDataWPOLM")
CSNegDataFinalPoint = spark.sql("select sum(totalpoints) as finalpoints from AllCSNegDataWPOLM")
CSNegDataFinalPoint = CSNegDataFinalPoint.take(1)
CSNegScore = round(CSNegDataFinalPoint[0][0], 3)
print(CSNegScore)

23.707


In [187]:
finalscores.append((cityName, fitnessStudioType, "pos", CSPosScore))
finalscores.append((cityName, fitnessStudioType, "neg", CSNegScore))
avgscores.append((cityName, fitnessStudioType, round( ((CSPosScore + CSNegScore)/2), 3)))

In [188]:
key = "" 
for items in finalscores:
    if key != items[0]:
        print("-----")
        key = items[0]
    print(items)

-----
('Tempe', 'Gym', 'pos', 29.397)
('Tempe', 'Gym', 'neg', 24.71)
('Tempe', 'Yoga', 'pos', 28.576)
('Tempe', 'Yoga', 'neg', 23.186)
('Tempe', 'Martial Arts', 'pos', 26.482)
('Tempe', 'Martial Arts', 'neg', 22.002)
-----
('Cleveland', 'Martial Arts', 'pos', 23.446)
('Cleveland', 'Martial Arts', 'neg', 19.01)
('Cleveland', 'Gym', 'pos', 26.55)
('Cleveland', 'Gym', 'neg', 23.009)
('Cleveland', 'Yoga', 'pos', 29.293)
('Cleveland', 'Yoga', 'neg', 19.474)
-----
('Las Vegas', 'Yoga', 'pos', 29.448)
('Las Vegas', 'Yoga', 'neg', 22.373)
('Las Vegas', 'Martial Arts', 'pos', 28.153)
('Las Vegas', 'Martial Arts', 'neg', 22.176)
-----
('Phoenix', 'Gym', 'pos', 26.213)
('Phoenix', 'Gym', 'neg', 28.371)
-----
('Pittsburg', 'Gym', 'pos', 26.74)
('Pittsburg', 'Gym', 'neg', 27.594)
-----
('Phoenix', 'Yoga', 'pos', 28.234)
('Phoenix', 'Yoga', 'neg', 28.132)
-----
('Pittsburg', 'Yoga', 'pos', 26.689)
('Pittsburg', 'Yoga', 'neg', 27.143)
-----
('Las Vegas', 'Gym', 'pos', 28.627)
('Las Vegas', 'Gym', 'ne

In [189]:
#thus looking at the scores, we can decide which studio is preffered in which city
key = ""
for items in sorted(avgscores, key = lambda x: (x[0], x[2]), reverse=True):
    if key != items[0]:
        print("-----")
        key = items[0]
    print(items)
    

-----
('Tempe', 'Gym', 27.053)
('Tempe', 'Yoga', 25.881)
('Tempe', 'Martial Arts', 24.242)
-----
('Pittsburg', 'Gym', 27.167)
('Pittsburg', 'Yoga', 26.916)
-----
('Phoenix', 'Yoga', 28.183)
('Phoenix', 'Gym', 27.292)
-----
('Las Vegas', 'Gym', 26.167)
('Las Vegas', 'Yoga', 25.91)
('Las Vegas', 'Martial Arts', 25.164)
-----
('Cleveland', 'Gym', 24.779)
('Cleveland', 'Yoga', 24.383)
('Cleveland', 'Martial Arts', 21.228)


In [191]:
key = "" 
for items in sorted(finalscores, key = lambda x: (x[0], x[3]), reverse=True):
    if key != items[0]:
        print("-----")
        key = items[0]
    print(items)

-----
('Tempe', 'Gym', 'pos', 29.397)
('Tempe', 'Yoga', 'pos', 28.576)
('Tempe', 'Martial Arts', 'pos', 26.482)
('Tempe', 'Gym', 'neg', 24.71)
('Tempe', 'Yoga', 'neg', 23.186)
('Tempe', 'Martial Arts', 'neg', 22.002)
-----
('Pittsburg', 'Gym', 'neg', 27.594)
('Pittsburg', 'Yoga', 'neg', 27.143)
('Pittsburg', 'Gym', 'pos', 26.74)
('Pittsburg', 'Yoga', 'pos', 26.689)
-----
('Phoenix', 'Gym', 'neg', 28.371)
('Phoenix', 'Yoga', 'pos', 28.234)
('Phoenix', 'Yoga', 'neg', 28.132)
('Phoenix', 'Gym', 'pos', 26.213)
-----
('Las Vegas', 'Yoga', 'pos', 29.448)
('Las Vegas', 'Gym', 'pos', 28.627)
('Las Vegas', 'Martial Arts', 'pos', 28.153)
('Las Vegas', 'Gym', 'neg', 23.707)
('Las Vegas', 'Yoga', 'neg', 22.373)
('Las Vegas', 'Martial Arts', 'neg', 22.176)
-----
('Cleveland', 'Yoga', 'pos', 29.293)
('Cleveland', 'Gym', 'pos', 26.55)
('Cleveland', 'Martial Arts', 'pos', 23.446)
('Cleveland', 'Gym', 'neg', 23.009)
('Cleveland', 'Yoga', 'neg', 19.474)
('Cleveland', 'Martial Arts', 'neg', 19.01)
